In [1]:
from neo4j import GraphDatabase
from datetime import  datetime
import numpy as np
import pandas as pd

In [2]:
uri= "neo4j://localhost:7687"
driver = GraphDatabase.driver(uri, auth=( "neo4j", "1234"))
session = driver.session()

In [3]:
Time_Dictionary=dict()
average_dict=dict()
stdv_dic=dict()

# FUNCTION

In [4]:
def calculate_time(query,variable_name):
    start_time = datetime.now()
    session.run(query)
    end_time = datetime.now()
    time=(round((end_time- start_time ).total_seconds()*1000))
    add_to_list(variable_name,time)   

def add_to_list(variable_name,time):
    if variable_name in Time_Dictionary:
        Time_Dictionary[variable_name].append(time)
    else:
        Time_Dictionary.update({variable_name:[]})
        Time_Dictionary[variable_name].append(time) 
        
def change_name_index(value):
    value=value+1
    result= 'round '+ str(value)
    return result
    
def create_dataframe(dictionary):
    for key in dictionary:
        average_dict.update({key:[]})
        average_dict[key].append(sum(dictionary[key])/len(dictionary[key]))
    for key in dictionary:
        stdv_dic.update({key:[]})
        stdv_dic[key].append(np.std(dictionary[key]))
    result = pd.DataFrame.from_dict(dictionary)
    for index, row in result.iterrows():
        result.rename(index={index:change_name_index(index)},inplace=True)
    average=pd.DataFrame.from_dict(average_dict)
    average.index=['Average']
    stdv=pd.DataFrame.from_dict(stdv_dic)
    stdv.index=['stdv']
    frames=[result, average, stdv]
    final_result=pd.concat(frames)
    write_csv(final_result)
    
def write_csv(result):
    result.to_csv('cypher.csv')  

In [6]:
print(session.run("""MATCH(n)
RETURN count(n)
""").data())

[{'count(n)': 559600}]


In [5]:
Start_Proces=datetime.now()

In [ ]:
for i in range(10):
    print('round '+ str(i+1)+ ' started')
    query1 = """MATCH (n1)  
            WHERE n1.name='XU JIE'  
            RETURN n1.node_id as id, head(labels(n1)) as lbl
            """
    calculate_time(query1,"query1")

    query2="""MATCH(n) WHERE n.name='EL PORTADOR' and n.countries<>'' 
         RETURN  HEAD(LABELS(n)) as lbl , n.countries as country 
            """
    calculate_time(query2,"query2")
    
    query3="""MATCH (n)  
            RETURN DISTINCT HEAD(LABELS(n)) as lbl 
    """
    calculate_time(query3,"query3")

    query4="""
        MATCH (n)-[r1]->(n1)  
        RETURN DISTINCT TYPE(r1) as lbl
    """
    calculate_time(query4,"query4")
    
    query5="""MATCH (n1)-[r1:intermediary_of]->(n2)  
        WHERE n1.name='ORION HOUSE SERVICES (HK) LIMITED' 
        RETURN n2.name as name, HEAD(LABELS(n2)) as lbl 
        """
    calculate_time(query5,"query5")
    
    query6="""MATCH (n1)-[r1:officer_of]->(n2) 
              WHERE n1.name='BEARER' 
              RETURN HEAD(LABELS(n1)) as lbl1, 
              n2.name as name, r1.link as link, HEAD(LABELS(n2)) as lbl2 
                UNION ALL 
                MATCH(n1)-[r1:officer_of]->(n2) 
                              WHERE n1.name='The Bearer' 
                              RETURN HEAD(LABELS(n1)) as lbl1, 
                              n2.name as name, r1.link as link, HEAD(LABELS(n2)) as lbl2            
            """
    calculate_time(query6,"query6")
    
    query7="""MATCH (n1)-[r1]->(n2) WHERE r1.end_date='11-APR-2014' and n2.status='Active' and n1.name='THE BEARER' 
                RETURN HEAD(LABELS(n1)) as lbl1, r1.link as link, HEAD(LABELS(n2)) as lbl2, n2.name as entity_name 
                UNION ALL 
                MATCH (n1)-[r1]->(n2) 
                WHERE r1.end_date='29-DEC-2009' and n2.status='Defaulted' and n1.name='THE BEARER' 
                RETURN HEAD(LABELS(n1)) as lbl1, r1.link as link, HEAD(LABELS(n2)) as lbl2, n2.name as entity_name
              """
    calculate_time(query7,"query7")  

    query8="""MATCH (n1)-[r1:registered_address]->(n3)  
                RETURN HEAD(LABELS(n1)) as lbl1, n1.name as name, r1.link as link, n3.address as address 
              """
    calculate_time(query8,"query8")

    query9="""
            MATCH(n1)-[r1:registered_address]->(n2), 
             (n1)-[r2:registered_address]->(n3) 
        RETURN HEAD(LABELS(n1)) as lbl, n1.name as name, r1.link as link1, n2.address as first_address, r2.link as link2, n3.address as second_address 
              """
    calculate_time(query9,"query9")
    
    query10="""MATCH(n1:officer)-[r1:registered_address]->(n3)<-[r2:registered_address]-(n2:entity) 
                RETURN r1.link as link1, r2.link as link2, n3.address as address 
                """
    calculate_time(query10,"query10")

    query11="""MATCH(n1:intermediary)-[r1]->(n2:entity)-[r2]->(n3:address)<-[r3]-(n4:officer) 
            RETURN n1.name as name, r1.valid_until as valid, r1.sourceID as sourceID
            """
    calculate_time(query11,"query11")

    query12=""" MATCH (n1:officer)-[r1:officer_of]->(n2:officer)-[r2:registered_address]->(n3:address), (n2)-[r3:officer_of]->(n4:entity)<-[r4:intermediary_of]-(n5:intermediary)-[r5:intermediary_of]->(n6:entity)      
            RETURN n1.name as officer_name, r1.link as link1, n2.name as officer2_name, r3.link as link2 , n4.name as entity_name, n5.name as intermediary_name, r4.valid_until as valid_until, n6.name as e_name               
            """
    calculate_time(query12,"query12")
    print('round '+ str(i+1)+ ' finished')
    

In [10]:
End_Process=datetime.now()

In [11]:
Total_Time_Process=End_Process-Start_Proces

In [13]:
create_dataframe(Time_Dictionary)

In [14]:
print(round((Total_Time_Process ).total_seconds()*1000))

141480
